# Build a Simple LLM Application with LCEL(Langchain Expression Language)

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. 
This is a relatively simple LLM application - it's just a single LLM call plus some prompting. 
Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

In this Notebook:

- Using language models
- Using PromptTemplates and OutputParsers
- Using LangChain Expression Language (LCEL) to chain components together
- Debugging and tracing your application using LangSmith
- Deploying your application with LangServe

## Groq 

### Language Processing Unit Groq builds fast AI inference. 

The Groq LPU™, AI Inference Technology, delivers exceptional compute speed, affordability, and energy efficiency at scale.  Groq solutions are based on the Language Processing Unit (LPU), a new category of processor. Groq is the creator of the LPU and built it from the ground up to meet the unique characteristics and needs of AI. LPUs run Large Language Models (LLMs) at substantially faster speeds and, on an architectural level, up to 10x better energy efficiency compared to GPUs.

This paper explains the design principles of the Groq LPU and why its architecture delivers such exceptional performance.
chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://groq.com/wp-content/uploads/2024/07/GroqThoughts_WhatIsALPU-vF.pdf 

In [ ]:
!pip install langchain

- Environment Setup: Loads API keys from environment variables for OpenAI and Groq services.
    - Create a text file named `.env` in your project root folder and add your API keys like:
        ```
        OPENAI_API_KEY= "your-key-here"
        GROQ_API_KEY= "your-key-here"
        ```


In [13]:
# Load API keys from .env file and initialize OpenAI and Groq clients

import os
from dotenv import load_dotenv

load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

- LangSmith Configuration: Sets up environment variables for tracking and monitoring LangChain operations in LangSmith platform.

In [17]:
# Configure LangSmith tracking with API key and project settings

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [ ]:
!pip install langchain_groq

- Model Initialization: Sets up a chat model using Groq's Gemma-2-9b-it model with the specified API key.
    - ChatGroq: A LangChain integration that allows using Groq's language models through their API, providing access to models like Gemma-2-9b for chat completions.

In [18]:
# Initialize chat models from OpenAI and Groq providers

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000222AE1B8CE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000222AE1B9340>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
!pip install langchain_core

- Message Chain Creation: Sets up a conversation with system and user messages for language translation task.

In [19]:
# Create message chain for translation and invoke the model

from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [20]:
result

AIMessage(content='Bonjour, comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 21, 'total_tokens': 32, 'completion_time': 0.02, 'prompt_time': 0.000202349, 'queue_time': 0.013323771000000002, 'total_time': 0.020202349}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-748a5716-3dd5-4afa-8f20-2d2d33874c34-0', usage_metadata={'input_tokens': 21, 'output_tokens': 11, 'total_tokens': 32})

- Output Parsing: Converts the model's response into a plain string format for easier handling.
    - StrOutputParser: A simple parser in LangChain that converts model outputs into plain text strings, removing any special formatting or metadata.

In [21]:
# Parse model output to string format

from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

- LCEL Pipeline: Creates a simple chain by combining the model and parser using LangChain Expression Language.

In [22]:
# Create and execute a simple LCEL chain combining model and parser

chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal option and suitable for addressing someone you don\'t know well or someone older than you.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual greeting, suitable for friends and family.)\n* **Coucou, comment ça va ?** (This is a very informal greeting, often used with close friends.)\n\nLet me know if you\'d like more options or have any other phrases you\'d like translated!\n'

- Prompt Template Creation: Sets up a reusable chat prompt structure for translation tasks with language and text variables.
    - ChatPromptTemplate: A LangChain class that creates structured templates for chat interactions, allowing dynamic input of variables like language and text into predefined message formats.

In [24]:
# Create a flexible prompt template for translations

from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [27]:
result=prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [28]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

- LCEL Chain Construction: Combines prompt template, model, and parser into a single pipeline for streamlined translation tasks.
    - LCEL Chain: A complete processing pipeline that strings together multiple components (prompt template, model, and parser) using the pipe operator to handle translation requests in a single step.

In [ ]:
# Create full translation chain: prompt -> model -> parser

chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

* **LCEL Components Guide**: overview of components that can be added to LangChain pipelines for enhanced functionality.

* **Main Components**:

1. **Retriever**
   - Purpose: Finds relevant documents from knowledge base for context-based responses
   - Usage: `chain = retriever | prompt | model | parser`

2. **Advanced Output Parsers**
   - Purpose: Converts responses into specific formats (JSON, tables)
   - Usage: `parser = StructuredOutputParser()`

3. **Memory**
   - Purpose: Stores conversation history and past interactions
   - Usage: `chain = memory | retriever | prompt | model | parser`

4. **Tools**
   - Purpose: Enables model to access external tools (calculators, APIs)
   - Usage: `chain = tool | prompt | model | parser`

5. **Subchains**
   - Purpose: Combines multiple chains for complex workflows
   - Usage: `main_chain = chain_1 | chain_2`

6. **Callback Handlers**
   - Purpose: Monitors and logs model responses
   - Usage: `chain = prompt | model | parser | callback`

7. **Error Handlers**
   - Purpose: Manages and catches errors in the chain
   - Usage: `chain = error_handler | prompt | model | parser`

8. **Post-Processing**
   - Purpose: Modifies or analyzes model outputs
   - Usage: `chain = prompt | model | parser | post_process`

* **Key Benefit**: These components can be combined to create customized workflows based on your application's needs.

In [ ]:
# END